# Twitter Data Analysis

In this project, we will analyse the tweets with the most trending hashtag in New Delhi.

So there are 3 steps involved in this process. 

1. Getting the data from Twitter

2. Present the data in a proper format so that it can be analysed easily

3. Analyse the data

We will be working with python and Twitter API. 


## Extracting data from twitter


I have used Twitter API tweepy for doing this. So, we will start with installing tweepy. 

In [ ]:
pip install tweepy

Now, make a developer account on twitter. You might need to wait for some time before it gets approved
 
In the developer account click on "Create a new App" and you will be given credentials. 

Now , we will include those credentials to make GET requests from twitter api.

In [ ]:
consumer_key= 'YBA2m8JJ9cy3E9wXxVi2yYcX8'
consumer_secret= 'HFMeoWIOhDZAjbDpfgAyujxSSZEQI5BAOuGxPiQRT4HniPk9cq'
access_token= '1342378688453648384-tVtbtEATUh3NpZWttbaoKFp438NXpI'
access_token_secret= 'FTvj3KU0oJCe9ZJBGGnKDAMPhU6eICcqV7GqfUGu1a68Y'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api=tweepy.API(auth)

Now, to avoid running our program repetitively, we will want to store all the tweets in a file. So we will create a new text file "json_dumps.txt"

In [ ]:
import os
import json
import sys
import tweepy
import io

File = io.open('json_dumps.txt', 'w', encoding="utf-8")


Now we will make the GET request. Note three things:

1. We are using the hashtag "cricket" as, without retweets we can get more than 10000 tweets.  

2. We will be using a filter for retweets(for uniqueness)


In [ ]:
tweet_data =  tweepy.Cursor(api.search,q="#cricket"+" -filter:retweets",geocode="28.7041,77.1025,100km", count=11000,lang="en").items(11000)
#tweet_data does extract the top trending hashtag win a particular location and works well but here we want a large number of tweets
#(> 1000). So for now I have considered a really popular topic and location to be anywhere. 


tweet_data =  tweepy.Cursor(api.search,q="#cricket"+" -filter:retweets", count=11000,lang="en").items(11000)

We will be storing the tweets in a csv file as well. For this we will use pandas.So , using open() we create a new csv file and then  import pandas and create a dataframe.



In [ ]:
pip install pandas

In [ ]:

import pandas as pd
df = pd.DataFrame()

csv_file = io.open("tweets_info.csv", "w", encoding="utf-8")


Now , we will iterate through all the tweets in the tweet_data object. Every tweet has a _json_ attribute to it that contains all the info about the tweet. So we will write the json strings to the text file and the csv file.

In [ ]:
i = 0;
for tweet in  tweet_data:    
    if (hasattr(tweet , "_json")):
        i += 1;
        File.write(json.dumps(tweet._json))
        File.write("\n")
        df_2 = pd.json_normalize(tweet._json)
        df = df.append(df_2, ignore_index=True)
        print(i)
        if (i >=11000):
            print(i)
            break;
export_csv = df.to_csv(csv_file, index = None, header=True)

In [ ]:
print(i)

Voila!! We have got all the tweets and we have got them in a suitable format too 

##  Analysis of  the data.  

Now we have got all the stuff in csv file. So let's start with making a wordcloud for all the tweets. 
Start with getting text of all the posts in the tweets. For this, to pevent the need of repititive computation, we will first store all the words of the posts in a text file.

My initial approach was iterating through each tweet, extracting it's post text as a string and then split the string with spaces to get tokens. But the process is too slow and time taking. So for this, we will use spacy module in python.

So start with installing it through pip on your system. 

In [ ]:
pip install spacy

Import all required modules first. And then read the info stored in the tweets.text column in the csv file into a dataframe. 


In [ ]:
import os
import spacy
import io
from spacy.lang.en import English
# Use it on Jupyter Notebook or Google Colab
# DIR_PATH = os.getcwd()
# Use it on Python module
DIR_PATH = os.path.dirname("__file__")

FILE_PATH = os.path.join(DIR_PATH, "tweet_info.csv")

import pandas as pd

# Read the file
df = pd.read_csv(FILE_PATH)

Now iterate through all rows in the dataframe and save all the individual words in a list. 

In [ ]:
final_list = []
for i in range(11000):
    first_dialogue = df.loc[i, "text"]
    if first_dialogue == Nonedependency:
        break;
    # use spacy with the  parse 
    nlp = spacy.load("en_core_web_sm")
    [str(sent) for sent in nlp(first_dialogue).sents]

    
    # use spacy with the sentencizer
    nlp = English()  # just the language with no model
    sentencizer = nlp.create_pipe("sentencizer")
    nlp.add_pipe(sentencizer)
    k = [str(sent) for sent in nlp(first_dialogue).sents]
    for K in k:
        final_list.append(K)
        print(i)

Now save this list in a text file for future analysis. 

In [ ]:
with io.open("list_of_posts.txt", "w", encoding="utf-8") as f:
    f.write((' '.join([str(elem) for elem in final_list])))

Now, we can start with making a word cloud for all the posts related to the hashtag cricket. 

Start with importing all the files. 

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
import io


Store the data of the posts in a string. 

In [ ]:
with io.open("list_of_posts.txt", "r", encoding="utf-8") as file:
    data = file.read().replace('\n', '')

Now use the wordcloud and pyplot module to display the wordCloud. 

from wordcloud import WordCloud, STOPWORDS 
stopwords = set(STOPWORDS) 
wordcloud = WordCloud(width = 800, height = 800, background_color ='white', stopwords = stopwords, min_font_size = 10).generate(data) 

plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 


Now, lets analyse make a word cloud for the description of the users who tweeted related to cricket. We will plot histograms for the same. 

So we will repeat the same process as before. 

In [ ]:
import os
import spacy
import io
from spacy.lang.en import English
# Use it on Jupyter Notebook or Google Colab
# DIR_PATH = os.getcwd()
# Use it on Python module
DIR_PATH = os.path.dirname(__file__)

FILE_PATH = os.path.join(DIR_PATH, "tweet_info.csv")

import pandas as pd

# Read the file
df = pd.read_csv(FILE_PATH)
# Assign first_dialogue to the first row's "Dialogue" column
final_list = []
for i in range(1000):
	first_dialogue = df.loc[i, "user.description"]
	print(i)
	print(first_dialogue)
	# if first_dialogue == None:
	# 	break;	
	# use spacy with the dependency parse 
	nlp = spacy.load("en_core_web_sm")
	#[str(sent) for sent in nlp(first_dialogue).sents]

	
	# use spacy with the sentencizer
	nlp = English()  # just the language with no model
	sentencizer = nlp.create_pipe("sentencizer")
	nlp.add_pipe(sentencizer)
	final_list.append(first_dialogue)
	# try:
	# 	if (not(first_dialogue == nan)):
	# 		k = [str(sent) for sent in nlp(first_dialogue).sents]
	# 		print(k)
	# 		for K in k:
	# 			final_list.append(K)
	# 		print(i)
	# except:
	# 	print("Ok")
with io.open("list_of_user_description.txt", "w", encoding="utf-8") as f:
    f.write((' '.join([str(elem) for elem in final_list])))


Now we will make the word cloud. 

In [ ]:
# Python program to generate WordCloud 

# importing all necessery modules 
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
import io

# Reads 'Youtube04-Eminem.csv' file 
with io.open("list_of_user_description.txt", "r", encoding="utf-8") as file:
    data = file.read().replace('\n', '')

comment_words = '' 
STOPWORDS.add("https")
STOPWORDS.add("RahulGandhi")
STOPWORDS.add("Rahul Gandhi")
STOPWORDS.add("Rahul")
STOPWORDS.add("will")
STOPWORDS.add("t")
STOPWORDS.add("co")
STOPWORDS.add("Please")
stopwords = set(STOPWORDS) 

# # iterate through the csv file 
# for tokens in data: 
	
# 	tokens = tokens.lower()
# 	print(tokens) 
	
# 	comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 800, height = 800, 
				background_color ='white', 
				stopwords = stopwords, 
				min_font_size = 10).generate(data) 

# plot the WordCloud image					 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 


Now we will analyse the sentiment of the posts(positive, negative and neutral) and will present the findings with a bar chart. We have used a textblob module for this.  


In [ ]:
from textblob import TextBlob
import os
import spacy
import io
from spacy.lang.en import English
import matplotlib.pyplot as plt
# Use it on Jupyter Notebook or Google Colab
# DIR_PATH = os.getcwd()
# Use it on Python module
DIR_PATH = os.path.dirname(__file__)

FILE_PATH = os.path.join(DIR_PATH, "tweet_info.csv")

import pandas as pd

# Read the file
df = pd.read_csv(FILE_PATH)
# Assign first_dialogue to th
df["polarity"] =df['text'].apply(lambda tweet: TextBlob(tweet).polarity)

print(df["polarity"])

positive  = 0
negative = 0
neutral = 0;

for dub in df["polarity"]:
	print(dub)
	if dub > 0:
		positive += 1;
	if (dub< 0):
		negative += 1;
	if (dub == 0):
		neutral += 1;

label1 = ["positive", "negative", "neutral"]
label2 = [positive, negative, neutral]


fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

ax.bar(label1, label2)
plt.show()

from matplotlib import pyplot as plt 
import numpy as np 
  
  
# Creating dataset 
# a = np.array(df["user.followers_count"]) 
  
# # Creating histogram 
# fig, ax = plt.subplots(figsize =(10, 7),  
#                         tight_layout = True) 
# ax.hist(a, bins = 1000 ,
#                    color='#607c8e') 
  
# # Show plot 
# plt.show() 

a = np.array(df["user.followers_count"])

print(df["user.followers_count"]) 
# Creating histogram 
fig, ax = plt.subplots(figsize =(100, 70)) 
# cks = np.arange(0, 10000000, 100)x_ti
# plt.xticks(x_ticks)
ax.hist(a, bins = [0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000])
ax.set_xticklabels([0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000], rotation=0, fontsize=10)


# ax.set_xlabel('marks')
# ax.set_ylabel('no. of students') 


# Show plot 
plt.show() 

from matplotlib import pyplot as plt 
import numpy as np 
  
  
# Creating dataset 
# a = np.array(df["user.followers_count"]) 
  
# # Creating histogram 
# fig, ax = plt.subplots(figsize =(10, 7),  
#                         tight_layout = True) 
# ax.hist(a, bins = 1000 ,
#                    color='#607c8e') 
  
# # Show plot 
# plt.show() 

a = np.array(df["user.followers_count"])

print(df["user.friends_count"]) 
# Creating histogram 
fig, ax = plt.subplots(figsize =(100, 70)) 
# cks = np.arange(0, 10000000, 100)x_ti
# plt.xticks(x_ticks)
ax.hist(a, bins = [0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000])
ax.set_xticklabels([0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000], rotation=0, fontsize=10)


# ax.set_xlabel('marks')
# ax.set_ylabel('no. of students') 


# Show plot 
plt.show() 

Now we will make a wordCloud of the tags associated with cricket that have been used. 

In [ ]:
import os
import spacy
import io
from spacy.lang.en import English
# Use it on Jupyter Notebook or Google Colab
# DIR_PATH = os.getcwd()
# Use it on Python module
DIR_PATH = os.path.dirname(__file__)

FILE_PATH = os.path.join(DIR_PATH, "tweet_info.csv")

import pandas as pd

# Read the file
df = pd.read_csv(FILE_PATH)
# Assign first_dialogue to the first row's "Dialogue" column
final_list = []
for i in range(1000):
	first_dialogue = df.loc[i, "entities.hashtags"]
	print(i)
	print(first_dialogue)
	# if first_dialogue == None:
	# 	break;	
	# use spacy with the dependency parse 
	nlp = spacy.load("en_core_web_sm")
	#[str(sent) for sent in nlp(first_dialogue).sents]

	
	# use spacy with the sentencizer
	nlp = English()  # just the language with no model
	sentencizer = nlp.create_pipe("sentencizer")
	nlp.add_pipe(sentencizer)
	final_list.append(first_dialogue)
	# try:
	# 	if (not(first_dialogue == nan)):
	# 		k = [str(sent) for sent in nlp(first_dialogue).sents]
	# 		print(k)
	# 		for K in k:
	# 			final_list.append(K)
	# 		print(i)
	# except:
	# 	print("Ok")
with io.open("list_of_hashtags_associated_with_tweets.txt", "w", encoding="utf-8") as f:
    f.write((' '.join([str(elem) for elem in final_list])))

Now we need to use wordCloud. 

In [ ]:
# Python program to generate WordCloud 

# importing all necessery modules 
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
import io

# Reads 'Youtube04-Eminem.csv' file 
with io.open("list_of_hashtags_associated_with_tweets.txt", "r", encoding="utf-8") as file:
    data = file.read().replace('\n', '')

comment_words = '' 
STOPWORDS.add("https")
STOPWORDS.add("RahulGandhi")
STOPWORDS.add("Rahul Gandhi")
STOPWORDS.add("Rahul")
STOPWORDS.add("will")
STOPWORDS.add("t")
STOPWORDS.add("co")
STOPWORDS.add("Please")
STOPWORDS.add("text")
STOPWORDS.add("indices" '')
stopwords = set(STOPWORDS) 

# # iterate through the csv file 
# for tokens in data: 
	
# 	tokens = tokens.lower()
# 	print(tokens) 
	
# 	comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 800, height = 800, 
				background_color ='white', 
				stopwords = stopwords, 
				min_font_size = 10).generate(data) 

# plot the WordCloud image					 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 


Let's try to understand how often tweets related to cricket are retweeted, as while extracting the data, we were filtering out retweets to maintain uniqueness. 

Now to analyse users, lets first understand what is the distrubution of number of followers each user has vs distribution of number of people being followed by the users. 

In [ ]:
from matplotlib import pyplot as plt 
import numpy as np 
  
  
# Creating dataset 
# a = np.array(df["user.followers_count"]) 
  
# # Creating histogram 
# fig, ax = plt.subplots(figsize =(10, 7),  
#                         tight_layout = True) 
# ax.hist(a, bins = 1000 ,
#                    color='#607c8e') 
  
# # Show plot 
# plt.show() 

a = np.array(df["user.followers_count"])

print(df["user.followers_count"]) 
# Creating histogram 
fig, ax = plt.subplots(figsize =(100, 70)) 
# cks = np.arange(0, 10000000, 100)x_ti
# plt.xticks(x_ticks)
ax.hist(a, bins = [0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000])
ax.set_xticklabels([0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000], rotation=0, fontsize=10)


# ax.set_xlabel('marks')
# ax.set_ylabel('no. of students') 


# Show plot 
plt.show() 

from matplotlib import pyplot as plt 
import numpy as np 
  
  
# Creating dataset 
# a = np.array(df["user.followers_count"]) 
  
# # Creating histogram 
# fig, ax = plt.subplots(figsize =(10, 7),  
#                         tight_layout = True) 
# ax.hist(a, bins = 1000 ,
#                    color='#607c8e') 
  
# # Show plot 
# plt.show() 

a = np.array(df["user.followers_count"])

print(df["user.friends_count"]) 
# Creating histogram 
fig, ax = plt.subplots(figsize =(100, 70)) 
# cks = np.arange(0, 10000000, 100)x_ti
# plt.xticks(x_ticks)
ax.hist(a, bins = [0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000])
ax.set_xticklabels([0, 250, 500, 1000,2500, 5000,10000,20000,30000,40000, 50000, 60000, 70000, 80000, 90000, 100000], rotation=0, fontsize=10)


# ax.set_xlabel('marks')
# ax.set_ylabel('no. of students') 


# Show plot 
plt.show() 

Make a word cloud of description of users to understand what walks of life cricket tweeters come from :D. 

In [ ]:
import os
import spacy
import io
from spacy.lang.en import English
# Use it on Jupyter Notebook or Google Colab
# DIR_PATH = os.getcwd()
# Use it on Python module
DIR_PATH = os.path.dirname(__file__)

FILE_PATH = os.path.join(DIR_PATH, "tweet_info.csv")

import pandas as pd

# Read the file
df = pd.read_csv(FILE_PATH)
# Assign first_dialogue to the first row's "Dialogue" column
final_list = []
for i in range(1000):
	first_dialogue = df.loc[i, "user.description"]
	print(i)
	print(first_dialogue)
	# if first_dialogue == None:
	# 	break;	
	# use spacy with the dependency parse 
	nlp = spacy.load("en_core_web_sm")
	#[str(sent) for sent in nlp(first_dialogue).sents]

	
	# use spacy with the sentencizer
	nlp = English()  # just the language with no model
	sentencizer = nlp.create_pipe("sentencizer")
	nlp.add_pipe(sentencizer)
	final_list.append(first_dialogue)
	# try:
	# 	if (not(first_dialogue == nan)):
	# 		k = [str(sent) for sent in nlp(first_dialogue).sents]
	# 		print(k)
	# 		for K in k:
	# 			final_list.append(K)
	# 		print(i)
	# except:
	# 	print("Ok")
with io.open("list_of_user_description.txt", "w", encoding="utf-8") as f:
    f.write((' '.join([str(elem) for elem in final_list])))

In [ ]:
# Python program to generate WordCloud 

# importing all necessery modules 
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
import io

# Reads 'Youtube04-Eminem.csv' file 
with io.open("list_of_user_description.txt", "r", encoding="utf-8") as file:
    data = file.read().replace('\n', '')

comment_words = '' 
STOPWORDS.add("https")
STOPWORDS.add("RahulGandhi")
STOPWORDS.add("Rahul Gandhi")
STOPWORDS.add("Rahul")
STOPWORDS.add("will")
STOPWORDS.add("t")
STOPWORDS.add("co")
STOPWORDS.add("Please")
stopwords = set(STOPWORDS) 

# # iterate through the csv file 
# for tokens in data: 
	
# 	tokens = tokens.lower()
# 	print(tokens) 
	
# 	comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 800, height = 800, 
				background_color ='white', 
				stopwords = stopwords, 
				min_font_size = 10).generate(data) 

# plot the WordCloud image					 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 


Lers plot the retweeting frequency. 